In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import os
import copy

from dgl.geometry import farthest_point_sampler
import open3d as o3d

from utils import fps, depth2fgpcd, pcd2pix, fps_np, downsample_pcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [31]:
env = 'mustar_bottle'
folder_path = f'ptcl_data/{env}'

In [32]:
pcd_all_list = []
extrinsic_matrixs = []
global_scale = 24

for i in range(1, 5):
    # load data
    cam_view = 'view_{}'.format(i)
    dir_path = os.path.join(folder_path, cam_view)
    
    # obs = np.load(os.path.join(dir_path, 'obs.npy'))
    # camera_intrinsic_params = np.load(os.path.join(dir_path, 'camera_intrinsic_params.npy')) # [fx, fy, cx, cy]
    camera_extrinsic_matrix = np.load(os.path.join(dir_path, 'camera_extrinsic_matrix.npy'))
    extrinsic_matrixs.append(camera_extrinsic_matrix)
    
    pcd = o3d.io.read_point_cloud(os.path.join(dir_path, 'fgpcd.pcd'))
    
    pcd_all_list.append(pcd)
    

In [33]:
o3d.visualization.draw_geometries(pcd_all_list)

In [5]:
pcd_all = o3d.geometry.PointCloud()
pcd_all = [pcd_all_list[0]]
for point_id in range(2):
    trans = extrinsic_matrixs[point_id] @ np.linalg.inv(extrinsic_matrixs[0])
    pcd_all_list[point_id].transform(trans)
    pcd_all.append(pcd_all_list[point_id])

In [6]:
o3d.visualization.draw_geometries(pcd_all)

In [15]:
o3d.visualization.draw_geometries([pcd_all_list[1]])

### Merge 2

In [26]:
source_0 = copy.deepcopy(pcd_all_list[0]) 
source_1 = copy.deepcopy(pcd_all_list[1])

In [27]:
extr_0 = extrinsic_matrixs[0].copy()
extr_1 = extrinsic_matrixs[1].copy()

In [28]:
o3d.visualization.draw_geometries([source_0, source_1])

In [29]:
source_0.transform(extr_0)

PointCloud with 82329 points.

In [30]:
o3d.visualization.draw_geometries([source_0, source_1])

In [15]:
source_0.transform(extr_0 @ np.linalg.inv(extr_1))

PointCloud with 82329 points.

In [16]:
o3d.visualization.draw_geometries([source_0, source_1])